# QUADRATIC_ASSIGNMENT

## Overview
The QUADRATIC_ASSIGNMENT function solves the quadratic assignment problem (QAP), a classic combinatorial optimization problem in operations research. The QAP models the assignment of a set of facilities to a set of locations with the goal of minimizing the total cost, which is determined by the flow between facilities and the distance between locations. Mathematically, the objective is to find a permutation $\pi$ that minimizes:

```math
\text{Cost} = \sum_{i=1}^n \sum_{j=1}^n F_{i,j} \cdot D_{\pi(i),\pi(j)}
```

This function uses `scipy.optimize.quadratic_assignment` to efficiently solve the QAP for small to medium-sized matrices. The QAP is widely used in facility layout, keyboard design, and other resource allocation problems. See the [scipy.optimize.quadratic_assignment documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.quadratic_assignment.html) and [scipy GitHub repository](https://github.com/scipy/scipy) for more details. This example function is provided as-is without any representation of accuracy.

## Usage
To use the QUADRATIC_ASSIGNMENT function in Excel, enter it as a formula in a cell, specifying your flow and distance matrices as 2D lists or Excel ranges:

```excel
=QUADRATIC_ASSIGNMENT(flow_matrix, distance_matrix)
```
- `flow_matrix` (2D list of float, required): The flow between facilities (square matrix).
- `distance_matrix` (2D list of float, required): The distances between locations (square matrix).

The function returns a 2D list of [facility, location] pairs representing the optimal assignment, or an error message string if the input is invalid.

## Examples

**Example 1: Assigning Facilities to Minimize Cost**

Assign 3 facilities to 3 locations to minimize total cost.

In Excel:
```excel
=QUADRATIC_ASSIGNMENT({0,5,2;5,0,3;2,3,0}, {0,2,3;2,0,1;3,1,0})
```
Expected output:

| Facility | Location |
|----------|----------|
| 0        | 2        |
| 1        | 0        |
| 2        | 1        |

**Example 2: Handling Invalid Input**

If the matrices are not square or not the same size, an error message is returned.

In Excel:
```excel
=QUADRATIC_ASSIGNMENT({0,1;1,0}, {0,2,3;2,0,1;3,1,0})
```
Expected output:

"Both matrices must be square and of the same size."

In [ ]:
import numpy as np
from scipy.optimize import quadratic_assignment as scipy_quadratic_assignment

def quadratic_assignment(flow_matrix, distance_matrix):
    """Solves the quadratic assignment problem (QAP) for given flow and distance matrices.

    Args:
        flow_matrix (list[list[float]]): 2D list representing the flow between facilities (square matrix).
        distance_matrix (list[list[float]]): 2D list representing the distances between locations (square matrix).

    Returns:
        list[list[int]]: 2D list of [facility, location] assignments, or error message string.

    This example function is provided as-is without any representation of accuracy.
    """
    flow = np.array(flow_matrix)
    dist = np.array(distance_matrix)
    if flow.ndim != 2 or dist.ndim != 2:
        return "Both inputs must be 2D lists (matrices)."
    if flow.shape != dist.shape or flow.shape[0] != flow.shape[1]:
        return "Both matrices must be square and of the same size."
    result = scipy_quadratic_assignment(flow, dist)
    if not hasattr(result, 'col_ind'):
        return str(result)
    assignment = [[int(i), int(j)] for i, j in enumerate(result.col_ind)]
    return assignment

In [ ]:
import ipytest
ipytest.autoconfig()
import pytest

demo_cases = [
    [
        [[0, 5, 2], [5, 0, 3], [2, 3, 0]],
        [[0, 2, 3], [2, 0, 1], [3, 1, 0]],
        [[0, 2], [1, 1], [2, 0]]
    ],
    [
        [[0, 1], [1, 0]],
        [[0, 2, 3], [2, 0, 1], [3, 1, 0]],
        "Both matrices must be square and of the same size."
    ],
    [
        [],
        [],
        "Both inputs must be 2D lists (matrices)."
    ]
]

def approx_equal(a, b, rel=0.05, abs_tol=1e-4):
    if isinstance(a, list) and isinstance(b, list):
        if all(isinstance(x, list) and isinstance(y, list) for x, y in zip(a, b)):
            return all(
                all(isinstance(xi, int) and isinstance(yi, int) and xi == yi for xi, yi in zip(xrow, yrow))
                for xrow, yrow in zip(a, b)
            )
    return a == b

@pytest.mark.parametrize("flow_matrix, distance_matrix, expected", demo_cases)
def test_demo_cases(flow_matrix, distance_matrix, expected):
    result = quadratic_assignment(flow_matrix, distance_matrix)
    print(f"test_demo_cases output: {result}")
    if isinstance(expected, str):
        assert isinstance(result, str) and expected.lower() in result.lower()
    else:
        assert approx_equal(result, expected)

def test_invalid_not_2d():
    result = quadratic_assignment([1, 2, 3], [1, 2, 3])
    assert isinstance(result, str) and ("2d" in result.lower() or "matrix" in result.lower())

ipytest.run('-s')

In [ ]:
# Interactive Demo
import gradio as gr

demo = gr.Interface(
    fn=quadratic_assignment,
    inputs=[
        gr.DataFrame(label="Flow Matrix", type="array", value=demo_cases[0][0]),
        gr.DataFrame(label="Distance Matrix", type="array", value=demo_cases[0][1]),
    ],
    outputs=gr.DataFrame(headers=["Facility", "Location"], label="Assignments", type="array"),
    examples=[[case[0], case[1]] for case in demo_cases],
    flagging_mode="never",
    fill_width=True,
)
demo.launch()